In [ ]:
import sys
sys.path.insert(0, '/home/brandon/dev/hoomd-blue/build')
sys.path.insert(0, '/home/brandon/dev/hoomdpy-addons')

In [ ]:
import datetime
import os

import numpy as np
from tqdm.notebook import tqdm

import gsd.hoomd
import rowan
import hoomd
from hoomd import md
from hoomdpy_addons.visualization import Notebook

FILENAME='lj-sim.gsd'

In [ ]:
!rm lj-sim.gsd

In [ ]:
box = hoomd.Box.cube(10.)
N_unitcells_per_dim = 9
Np = N_unitcells_per_dim ** 3
if not os.path.exists(FILENAME):
    snap = gsd.hoomd.Snapshot()
    snap.configuration.box = [*box.L, *box.tilts]
    xs = np.linspace(-box.Lx / 2,
                     box.Lx / 2,
                     N_unitcells_per_dim,
                     endpoint=False)
    positions = np.zeros((Np, 3))
    positions[:] = np.array(
        [a.ravel() for a in np.meshgrid(xs, xs, xs)]).T
    
    snap.particles.position = positions
    snap.particles.N = Np
    snap.particles.types = ['A']
    snap.particles.typeid = np.zeros(len(positions), dtype=int)
    with gsd.hoomd.open(FILENAME, 'xb') as trajectory:
        trajectory.append(snap)

In [ ]:
sim = hoomd.Simulation(hoomd.device.CPU(),
                       seed=59)
sim.create_state_from_gsd(FILENAME)
sim.state.thermalize_particle_momenta(hoomd.filter.All(), kT=1.0)

In [ ]:
kT = 4.0
lj = md.pair.LJ(nlist=md.nlist.Cell(), r_cut=2.5)
lj.params[('A', 'A')] = {'epsilon': 1. / kT,
                          'sigma': 1.}
dt = 0.005

nvt = md.methods.NVT(
    hoomd.filter.All(),
    kT=hoomd.variant.Ramp(1.0, 1 / (4 * kT), 50_000, 100_000),
    tau=100 * dt)
integrator = md.Integrator(dt=dt, methods=[nvt], forces=[lj])

In [ ]:
sim.operations += integrator

In [ ]:
gsd = hoomd.write.GSD(
    FILENAME,
    trigger=hoomd.trigger.And(
        [hoomd.trigger.Periodic(1_000),
         hoomd.trigger.After(50_000)]
    ),
    filter=hoomd.filter.All()
)
sim.operations += gsd

In [ ]:
box_resize = hoomd.update.BoxResize(
    trigger=10,
    variant=hoomd.variant.Ramp(0, 1, 100, 50_000),
    box1=sim.state.box,
    box2=hoomd.Box.cube((Np / 1.4142) ** (1. / 3.))
    )
sim.operations += box_resize

In [ ]:
logger = hoomd.logging.Logger(categories=['scalar', 'string'])
logger['Step'] = (lambda: sim.timestep, 'scalar')

def eta():
    timesteps_remaining = sim.final_timestep - sim.timestep
    seconds_remaining = timesteps_remaining / sim.tps
    return str(
        datetime.timedelta(seconds=seconds_remaining)
    )

logger['ETR'] = (eta, 'string')
table = hoomd.write.Table(
    trigger=10_000,
    logger=logger
)
sim.operations += table

In [ ]:
visual = hoomd.write.CustomWriter(trigger=500, action=Notebook())
sim.operations += visual

In [ ]:
sim.operations -= visual

In [ ]:
sim.run(150_000)